## Beautiful run plot rendering

This notebook shows how to create a Weave Board with W&B workspace-like behavior.

We group data along the x-axis into 250 bins, and exactly compute the min / max / avg of each bucket. Unlike the W&B workspace, this computation is exact because it happens on unsampled data in the Weave engine. Only the final grouped result is sent back to the UI.

You can zoom on any of the plots by dragging a box. All plots are zoom synchronized using Weave 2-way binding, and all plots automatically rebin for the new window.

We've also factored out a "groupby" variable so you can group by run parameters from the varbar. You can change it to 'id' or 'config.b' to try other groupings.

All of this behavior is complete defined by the code below.

_TODO: There is one major outstanding issue here: we're using the start of the bucket for the x-coordinate
instead of the midpoint! Fix this!_


In [ ]:
import weave
from weave.legacy.scripts import syndata

In [ ]:
# Generate synthetic run data for now
runs = weave.save(syndata.random_runs(10, 10000, 10), 'runs')
# Doing this unnest operation in the UI doesn't work (like in the varbar). I don't remember
# if the types are wrong, or what the specific situation is.
# TODO: Figure out why we have to do this!
unnested_runs = weave.save(weave.use(runs.unnest()), 'random_runs_unnested')
#unnested_runs

In [ ]:

def plot_for_metric(metric_name):
    return weave.legacy.weave.panels.BoardPanel(
            lambda runs, groupby, zoom_domain_x, bin_domain_x: weave.legacy.weave.panels.Plot(
                            runs,
                            series=[
                                # Use two plot series, one for the avg line, and one for
                                # min/max area. We could reduce duplication here a bit more
                                # by updating the Plot API to allow common features of series
                                # to be factored out.
                                weave.legacy.weave.panels.Series(
                                        runs,
                                    select_functions={
                                        'x': lambda row: row["history.step"].bin(
                                            weave.legacy.weave.ops.numbers_bins_equal(bin_domain_x, 250))["start"],
                                        'label': lambda row: row[groupby],
                                        'y': lambda row: row["history"][metric_name].avg()
                                    },
                                    groupby_dims=['x', 'label'],
                                    constants=weave.legacy.weave.panels.PlotConstants(
                                        mark='line'
                                    )
                                ),
                                weave.legacy.weave.panels.Series(
                                    runs,
                                    select_functions={
                                        'x': lambda row: row["history.step"].bin(
                                            weave.legacy.weave.ops.numbers_bins_equal(bin_domain_x, 250))["start"],
                                        'label': lambda row: row[groupby],
                                        'y': lambda row: row["history"][metric_name].min(),
                                        'y2': lambda row: row["history"][metric_name].max(),
                                    },
                                    groupby_dims=['x', 'label'],
                                    constants=weave.legacy.weave.panels.PlotConstants(
                                        mark='area'
                                    )
                                )
                            ],
                            domain_x=zoom_domain_x,
                        ),
            #layout=weave.legacy.weave.panels.BoardPanelLayout(x=x, y=0, w=12, h=6)
        )

board = weave.legacy.weave.panels.Board(
    vars={
        'runs': unnested_runs,
        'groupby': 'config.a',
        # This holds the zoom boundaries, and is two-way synchronized with the plot zooms
        'zoom_domain_x': None,
        # This is the extent of the step field in our data (the minimum and maxium values)
        'step_domain': lambda runs: weave.legacy.weave.ops.make_list(
            a=weave.legacy.weave.ops.numbers_min(runs['history']['step']),
            b=weave.legacy.weave.ops.numbers_max(runs['history']['step'])),
        # The "domain" used for binning, which is the zoom range if its non-null, otherwise
        # the step_extent
        'bin_domain_x': lambda zoom_domain_x, step_domain: zoom_domain_x.coalesce(step_domain)
    },
    # TODO: Use EachColumn to do this automatically for all metrics.
    panels=[
        plot_for_metric('metric0'),
        plot_for_metric('metric1'),
        plot_for_metric('metric2'),
        plot_for_metric('metric3'),
        plot_for_metric('metric4'),
        plot_for_metric('metric5'),
        plot_for_metric('metric6'),
        plot_for_metric('metric7'),
    ]
)

In [ ]:
board